In [0]:
import sys

sys.path.append("../include_utils/")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
import pysam
from collections import OrderedDict, namedtuple
import operator
import multiprocessing as mp
from hdfstorehelper import HDFStoreHelper
import dill
from scipy import stats
from IPython.display import display

samtools = "/home/cfriedline/gpfs/src/samtools-1.3/samtools"
bcftools = "/home/cfriedline/gpfs/src/bcftools-1.3/bcftools"
picard = "/home/cfriedline/gpfs/src/broadinstitute-picard-03a1d72/dist/picard.jar"
java = "/home/cfriedline/g/src/jdk1.8.0_92/bin/java"
perl = "/home/cfriedline/gpfs/opt/ActivePerl-5.18/bin/perl"

vcfutils = "perl /home/cfriedline/g/src/bcftools-1.3/vcfutils.pl"
vcftools = "/home/cfriedline/bin/vcftools"
bcftools = "/home/cfriedline/gpfs/src/bcftools-1.3/bcftools"
tabix = "/home/cfriedline/gpfs/src/htslib-1.3/tabix"
bgzip = "/home/cfriedline/gpfs/src/htslib-1.3/bgzip"


def setup_r():
    os.environ['R_HOME'] = '/home/cfriedline/g/R3/lib64/R'
    os.environ['LD_LIBRARY_PATH'] = "%s/lib:%s:%s" % (os.environ['R_HOME'], 
                                                   os.environ['LD_LIBRARY_PATH'],
                                                     "/home/cfriedline/lib64")

setup_r()
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
r = robjects.r

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
sys.version

In [0]:
imp_dir = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40"
imputed_vcf_gz = os.path.join(imp_dir, "isect_snps.recode.vcf.gz_sorted.vcf.gz")
imputed_vcf = os.path.join(imp_dir, "isect_snps.recode.vcf.gz_sorted.vcf")
vcf_files = [imputed_vcf_gz]

In [0]:
vcf_files

In [0]:
translation_df = pd.read_csv("translation_table.csv", sep="\t", index_col=0)
def get_translated_name(n):
    row = translation_df.ix[n.strip()]
    return "%s_%d_%d" % (row['pop'], row.indiv, row.dup)

names = [get_translated_name(x.strip()) for x in open("{}.012.indv".format(imputed_vcf_gz)).readlines()]
names = pd.DataFrame(names, columns=["s"])
names["p"] = names.s.apply(lambda x: x.split("_")[0])

In [0]:
names.to_csv(os.path.join(imp_dir, "structure_pops.txt"), sep="\t", header=False, index=False)

In [0]:
pgd = "/home/cfriedline/g/src/PGDSpider_2.1.0.3/PGDSpider2-cli.jar"
spid = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/vcf_to_structure.spid"

In [0]:
!$java -Xmx1024m -Xms512m -jar $pgd

## spid file

```
# spid-file generated: Wed Aug 03 13:31:31 EDT 2016

# VCF Parser questions
PARSER_FORMAT=VCF

# Only output SNPs with a phred-scaled quality of at least:
VCF_PARSER_QUAL_QUESTION=
# Select population definition file:
VCF_PARSER_POP_FILE_QUESTION=/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/structure_pops.txt
# What is the ploidy of the data?
VCF_PARSER_PLOIDY_QUESTION=DIPLOID
# Do you want to include a file with population definitions?
VCF_PARSER_POP_QUESTION=true
# Output genotypes as missing if the phred-scale genotype quality is below:
VCF_PARSER_GTQUAL_QUESTION=
# Do you want to include non-polymorphic SNPs?
VCF_PARSER_MONOMORPHIC_QUESTION=false
# Only output following individuals (ind1, ind2, ind4, ...):
VCF_PARSER_IND_QUESTION=
# Only input following regions (refSeqName:start:end, multiple regions: whitespace separated):
VCF_PARSER_REGION_QUESTION=
# Output genotypes as missing if the read depth of a position for the sample is below:
VCF_PARSER_READ_QUESTION=
# Take most likely genotype if "PL" or "GL" is given in the genotype field?
VCF_PARSER_PL_QUESTION=true
# Do you want to exclude loci with only missing data?
VCF_PARSER_EXC_MISSING_LOCI_QUESTION=false

# STRUCTURE Writer questions
WRITER_FORMAT=STRUCTURE

# Specify the locus/locus combination you want to write to the STRUCTURE file:
STRUCTURE_WRITER_LOCUS_COMBINATION_QUESTION=
# Do you want to include inter-marker distances?
STRUCTURE_WRITER_LOCI_DISTANCE_QUESTION=false
# Specify which data type should be included in the STRUCTURE file  (STRUCTURE can only analyze one data type per file):
STRUCTURE_WRITER_DATA_TYPE_QUESTION=SNP
# Save more specific fastSTRUCTURE format?
STRUCTURE_WRITER_FAST_FORMAT_QUESTION=true
```

In [0]:
!{java} -Xmx4096m -Xms4096m -jar {pgd} -inputfile {imputed_vcf} -outputfile {imp_dir}/structure.str -spid {spid}

In [0]:
strdf = pd.read_csv(os.path.join(imp_dir, "structure.str"), sep="\t", header=None)
strdf = strdf.sort_values(0)
strdf[0] = [get_translated_name(x) for x in strdf[0]]

In [0]:
strdf['p'] = strdf[0].apply(lambda x: x.split("_")[0])

In [0]:
poporder = {'NC':1, 'NY':4, 'QC32':5, 'QC93':6, 'VA1':2, 'VA2':3}

In [0]:
strdf['porder'] = strdf['p'].apply(lambda x: poporder[x])

In [0]:
strdf = strdf.sort_values(['porder', 0])

In [0]:
strdf.ix[:,0:7].to_csv(os.path.join(imp_dir, "structure_sorted.str"),
            sep="\t", 
            header=False,
            index=False)

In [0]:
strdf.head()

In [0]:
with open(os.path.join(imp_dir, "poplabels.txt"), "w") as o:
    for sample, data in strdf.groupby(0):
        o.write("{}\n".format(data['p'].values[0]))

## Setup jobs from 4-8

`conda create -n faststructure python=2.7 numpy scipy cython`

`$ cat faststructure.q`

```
#$ -S /bin/sh
#$ -N fS
#$ -cwd
#$ -V
source activate faststructure
python $HOME/g/src/fastStructure/structure.py -K $1 \
--input=structure_sorted \
--output=faststructure.$1.$SGE_TASK_ID \
--format=str \
--full
```

## Run jobs
```
qsub -t 1-5 faststructure.q "2"
qsub -t 1-5 faststructure.q "3"
qsub -t 1-5 faststructure.q "4"
qsub -t 1-5 faststructure.q "5"
qsub -t 1-5 faststructure.q "6"
qsub -t 1-5 faststructure.q "7"
qsub -t 1-5 faststructure.q "8"
#qsub -t 1-5 faststructure.q "9"
#qsub -t 1-5 faststructure.q "10"
```


## Summarize with PopHelper (https://github.com/royfrancis/pophelper)

```R
install.packages('devtools',dependencies=T)
library(devtools)
install_github('royfrancis/pophelper')
```

```R
library(pophelper)
files = list.files(path=".", pattern=glob2rx("*.meanQ"))
tr = tabulateRunsMatrix(files)
sr = summariseRunsMatrix(tr)
clumppExportMatrix(files, useexe=T)
poplabels = read.delim("poplabels.txt", header=F, stringsAsFactors=F)
collectClumppOutput(prefix="MATRIXpop", filetype="merged")
merged_files=list.files(path="MATRIXpop-merged", full.names=T)
plotRuns(merged_files, imgoutput="tab", poplab=poplabels$V1, subsetpops=c("NC", "VA1", "VA2", "NY", "QC32", "QC93"))
```

In [0]:
## separate out into run1, run2 to run chooseK.py

outprefix = os.path.join(imp_dir, "faststructure*.log")

import shutil
import glob

logfiles = glob.glob(outprefix)

for l in logfiles:
    chain = os.path.basename(l).split(".")[-3]
    rundir = "{}/run{}".format(os.path.join(os.path.dirname(l)), chain)
    if not os.path.exists(rundir):
        os.mkdir(rundir)
    shutil.copy(l, rundir)
    shutil.copy(l.replace(".log", ".meanQ"), rundir)

## Plotting

In [0]:
poplabels = pd.read_csv(os.path.join(imp_dir, "poplabels.txt"), header=None, names=["poplabel"])

In [0]:
# sed 's/^[ ]*//g' MATRIXpop_K7-combined-merged.txt | sed 's/: \+/ /g' | sed 's/ \+/ /g' > MATRIXpop_K7-combined-merged.txt.fixed
merged_dir = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/MATRIXpop-merged"
merged_file= "MATRIXpop_K7-combined-merged.txt.fixed"
merged = pd.read_csv(os.path.join(merged_dir, merged_file), sep=" ", header=None)
merged = merged.drop(0, axis=1)

In [0]:
labeled = merged.join(poplabels)

In [0]:
newcols = ["Cluster{}".format(x) for x in labeled.columns[0:7]]
newcols.append("summ")
newcols.append("poplabel")

In [0]:
newcols

In [0]:
labeled.columns = newcols

In [0]:
labeled.head()

In [0]:
labeled.to_csv("labeled.csv", sep="\t", header=True, index=True)

In [0]:
melted = pd.melt(labeled.drop("summ", axis=1), 
                 id_vars=['poplabel'],
                var_name=["cluster"])

In [0]:
melted.head()

In [0]:
num = 1
sns.set_context("talk")
for label, data in labeled.groupby("poplabel"):
    d = data.iloc[:,0:7].apply(np.mean)
    plt.subplot(2,3,num)
    patches, texts = plt.pie(d, shadow=False, startangle=0)
    plt.title(label)
    plt.axis('equal')
    plt.tight_layout()
    num+=1

    plt.legend(patches, list(d.index), 
           loc = 'upper right', 
           bbox_to_anchor = (0,-0.35,1.1,1),
           bbox_transform = plt.gcf().transFigure)
plt.show()

In [0]:
gps = {'QC32':[47.2509807, -79.4060515],
      'QC93': [46.90826, -70.8061075],
      'NC': [36.449125, -76.024672],
      'NY': [42.897768, -74.094761],
      'VA1': [38.657615, -77.463603],
      'VA2': [38.857470, -77.695003]}
gps_df = pd.DataFrame(gps).T
gps_df.columns = ['lat','lon']

In [0]:
if not os.path.exists("latlon.txt"):
    gps_df.to_csv("laton.txt", sep="\t", header=True, index=True)

In [0]:
from mpl_toolkits.basemap import Basemap

In [0]:
m = Basemap(projection='cyl', 
            llcrnrlon=-89.950078, 
            llcrnrlat=33.565319, 
            urcrnrlon=-69.977811, 
            urcrnrlat=47.912892, resolution=None)

In [0]:
m.readshapefile("/home/cfriedline/g/src/tl_2014_us_state/tl_2014_us_state", name="states", drawbounds=False);
m.readshapefile("/home/cfriedline/g/src/ca_province/PROVINCE", name="ca", drawbounds=False);

In [0]:
m.drawcoastlines()

In [0]:
p, t = plt.pie(d, shadow=False, startangle=0)
ax = plt.gca()
plt.axis("equal")